In [1]:
import string
import re
import collections
import csv
import pandas as pd
import numpy as np
import requests
import os
import json
from hyphen import Hyphenator

import dask.dataframe as dd
import multiprocessing

import matplotlib.pyplot as plt
from matplotlib.ticker import PercentFormatter

import seaborn as sns

%matplotlib inline

import stanfordnlp
from stanfordnlp.server import CoreNLPClient

import tensorflow.compat.v1 as tf
import tensorflow_hub as hub
#import tensorflow as tf


# Uncomment if needed to fix this error:
# OMP: Error #15: Initializing libiomp5md.dll, but found libiomp5md.dll already initialized.
os.environ["KMP_DUPLICATE_LIB_OK"]="TRUE"

In [2]:
PREDICTION_PATH = './predictions'
TEST_SETS_PATH = './test_sets'

#SET_NAMES = ['Amazon', 'Reddit', 'New-Wiki', 'NYT', 'dev-v1.1']
SET_NAMES = ['Amazon', 'Reddit', 'New-Wiki', 'NYT']

nlp = stanfordnlp.Pipeline(processors='tokenize', use_gpu=True)

Use device: gpu
---
Loading: tokenize
With settings: 
{'model_path': 'C:\\Users\\justi\\stanfordnlp_resources\\en_ewt_models\\en_ewt_tokenizer.pt', 'lang': 'en', 'shorthand': 'en_ewt', 'mode': 'predict'}
Done loading processors!
---


In [3]:
def load_data(input_file_path):
    with open(input_file_path, 'r', encoding='utf-8') as csv_file:
        csv_reader = csv.DictReader(csv_file)
        return [r for r in csv_reader]

In [4]:
df_answers = pd.read_csv('answers.csv')
df_distinct_answers = pd.read_csv('distinct_answers.csv')
df_distinct_context = pd.read_csv('distinct_context.csv')
df_merged_answers = pd.read_csv('merged_answers.csv')
df_merged_answers_and_context = pd.read_csv('merged_answers_and_context.csv')
df_pred_answers_context = pd.read_csv('pred_answers_context.csv')

df_pred = pd.DataFrame(load_data(PREDICTION_PATH + '/all_predictions.csv'))

C:\Users\justi\.conda\envs\w210_capstone\lib\site-packages\IPython\core\interactiveshell.py:3071: DtypeWarning: Columns (10,11,12) have mixed types.Specify dtype option on import or set low_memory=False.
  has_raised = await self.run_ast_nodes(code_ast.body, cell_name,


KeyboardInterrupt: 

In [ ]:
df_pred = df_pred.astype({'f1': 'float'})


In [ ]:
df_pred['f1'].hist(by=df_pred['test_set'],density=True)

In [ ]:
fig, axes = plt.subplots(1,4, figsize= (20,5))
fig.suptitle('Test Set F1 Histogram by %')
for i, test_set in enumerate(df_pred['test_set'].unique()):
    data = df_pred[df_pred['test_set'] == test_set]['f1']
    data.hist(ax=axes.flat[i], 
            bins=20, 
            weights=np.ones(len(data)) / len(data) 
           )
    axes.flat[i].set_title(test_set)
for ax in axes:
    ax.yaxis.set_major_formatter(PercentFormatter(1))

In [ ]:
fig, axes = plt.subplots(1,4, figsize= (20,5))
fig.suptitle('Test Set F1 Histogram by Count')
for i, test_set in enumerate(df_pred['test_set'].unique()):
    data = df_pred[df_pred['test_set'] == test_set]['f1']
    data.hist(ax=axes.flat[i], 
            bins=20
           )
    axes.flat[i].set_title(test_set)

In [ ]:
df_pred[df_pred['test_set'] == 'dev-v1.1']['f1'].hist(bins=1)

In [ ]:
fig, axes = plt.subplots(1,1)
axes.hist(df_pred[df_pred['test_set'] == 'Amazon']['f1'], 
          weights=np.ones(len(df_pred[df_pred['test_set'] == 'Amazon']['f1'])) / len(df_pred[df_pred['test_set'] == 'Amazon']['f1']) )
plt.show()

In [ ]:
df_distinct_context.sort_values('flesch-kincaid_grade_level', ascending=False).iloc[2]['context']

In [ ]:
df_distinct_context.iloc[1519]

In [ ]:
df_distinct_context.sort_values(
    'flesch-kincaid_grade_level', 
    ascending=False).iloc[2:3][[
                                'sentence_count', 
                                'word_count', 
                                'flesch-kincaid_grade_level', 
                                'coleman-liau', 
                                'gunning-fog', 
                                'automated-readability']]

In [ ]:
len(nlp(df_distinct_context.sort_values('flesch-kincaid_grade_level', ascending=False).iloc[2]['context']).sentences)

In [ ]:
df_distinct_context.sort_values('flesch-kincaid_grade_level', ascending=False)

In [ ]:
df_distinct_context.sort_values('flesch-kincaid_grade_level', ascending=False).iloc[1]

In [ ]:
x = 'University of the Philippines College of Medicine'.split()
for w in x:
    print(w, Hyphenator('en_US').syllables(w))

In [ ]:
df_merged_answers[df_merged_answers['question_id'] == '5dd469facc027a086d65bf1a']

In [ ]:
df_merged_answers[df_merged_answers['is_numeric']].groupby('first_parse').count()

In [ ]:
df_pred_answers_context[(df_pred_answers_context['test_set_x'] == 'Amazon') & (df_pred_answers_context['model_name'] == 'xlnet-123(singlemodel)')][['f1','syllables_per_word']].corr()

In [ ]:
fig, axes = plt.subplots(1,4, figsize= (22,5))

metric_list = ['f1','syllables_per_word', 'polysyllable_count','avg_word_length','avg_sentence_length_in_words','context_character_count',
                                          'avg_sentence_length_in_characters','flesch-kincaid_grade_level','coleman-liau','gunning-fog','automated-readability']
metric_list = ['f1', 'flesch-kincaid_grade_level','coleman-liau','gunning-fog','automated-readability']
for i, model in enumerate(SET_NAMES):
    corrMatrix = df_pred_answers_context[(df_pred_answers_context['test_set_x'] == model)][metric_list].corr()
#     print(model)
#    print(corrMatrix[0])
#     print('\n')
    
    p = sns.heatmap(corrMatrix, annot=True, ax = axes.flat[i], fmt=".2f")
    axes.flat[i].set_title(model)

    #p.set_xticklabels(p.get_xticklabels(), rotation = 0)
    
fig.tight_layout()
#plt.savefig('test.jpg')   

In [ ]:
fig, axes = plt.subplots(1,4, figsize= (22,5))

metric_list = ['exact_match','syllables_per_word', 'polysyllable_count','avg_word_length','avg_sentence_length_in_words','context_character_count',
                                          'avg_sentence_length_in_characters','flesch-kincaid_grade_level','coleman-liau','gunning-fog','automated-readability']
metric_list = ['exact_match', 'flesch-kincaid_grade_level','coleman-liau','gunning-fog','automated-readability']
for i, model in enumerate(SET_NAMES):
    corrMatrix = df_pred_answers_context[(df_pred_answers_context['test_set_x'] == model)][metric_list].corr()
#     print(model)
#    print(corrMatrix[0])
#     print('\n')
    
    p = sns.heatmap(corrMatrix, annot=True, ax = axes.flat[i], fmt=".2f")
    axes.flat[i].set_title(model)

    #p.set_xticklabels(p.get_xticklabels(), rotation = 0)
    
fig.tight_layout()
#plt.savefig('test.jpg') 

In [ ]:
pd.set_option('display.max_rows', 20)
df_pred_answers_context[df_pred_answers_context['context'] == df_distinct_context.sort_values('flesch-kincaid_grade_level', ascending=False).iloc[2]['context']][['question_text', 'answer_text', 'predicted_answer', 'f1', 'exact_match']].drop_duplicates()

In [ ]:
df_distinct_answers['answer_text'][0]

In [5]:
#import tensorflow as tf
import tensorflow.compat.v1 as tf
import tensorflow_hub as hub
#import tensorflow as tf

In [6]:
tf.test.is_gpu_available()

True

In [7]:
tf.config.list_physical_devices()

[PhysicalDevice(name='/physical_device:CPU:0', device_type='CPU'),
 PhysicalDevice(name='/physical_device:XLA_CPU:0', device_type='XLA_CPU'),
 PhysicalDevice(name='/physical_device:GPU:0', device_type='GPU'),
 PhysicalDevice(name='/physical_device:XLA_GPU:0', device_type='XLA_GPU')]

In [ ]:
from absl import logging

import tensorflow as tf

import tensorflow_hub as hub
import matplotlib.pyplot as plt
import numpy as np
import os
import pandas as pd
import re
import seaborn as sns

module_url = "https://tfhub.dev/google/universal-sentence-encoder/4" 
model = hub.load(module_url)
print ("module %s loaded" % module_url)
def embed(input):
  return model(input)

In [10]:
embed = hub.Module("https://tfhub.dev/google/universal-sentence-encoder/4")
embedding = embed(["The quick brown fox jumps over the lazy dog."])

FailedPreconditionError: Failed to remove a directory: C:\Users\justi\AppData\Local\Temp\tfhub_modules\063d866c06683311b44b4992fd46003be952409c.71e14574443d4663a4ba140e6dff25c0.tmp; Directory not empty

In [4]:
from tensorflow.python.client import device_lib
device_lib.list_local_devices()

[name: "/device:CPU:0"
 device_type: "CPU"
 memory_limit: 268435456
 locality {
 }
 incarnation: 12776218948778950073,
 name: "/device:XLA_CPU:0"
 device_type: "XLA_CPU"
 memory_limit: 17179869184
 locality {
 }
 incarnation: 9510981095795775933
 physical_device_desc: "device: XLA_CPU device",
 name: "/device:XLA_GPU:0"
 device_type: "XLA_GPU"
 memory_limit: 17179869184
 locality {
 }
 incarnation: 8786231927416512371
 physical_device_desc: "device: XLA_GPU device"]